## Convert KITTI Point Cloud to BEV image

In [ ]:
from pathlib import Path
from easydict import EasyDict
from tqdm import tqdm
import pickle
import yaml
import cv2

from pcdet.utils.bev_utils import get_sample, pointcloud3d_to_bevimage2d, show_bev_image_preview, generate_groundtruth_dicts_bev
from pcdet.utils.bev_utils import boxes3d_lidar_to_rotated_bev_boxes, draw_bev_boxes, save_bev_images_and_boxes, get_frame_ids

box_colormap = [
    [1, 1, 1], # not assigned
    [0, 1, 0], # Car (green)
    [1, 0, 1], # Pedestrian (violet)
    [0, 1, 1], # Cyclist (yellow)
]

dataset_cfg = EasyDict(yaml.safe_load(open('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/dataset_configs/kitti_dataset_bev_custom.yaml')))
class_names = ['Car', 'Pedestrian', 'Cyclist']
file_path = '/home/rlab10/OpenPCDet/lidar2bev/master_thesis/lidar3d_to_bev2d.ipynb' 
NOTEBOOK_DIR = (Path(file_path).resolve().parent).resolve()
ROOT_DIR = Path('/home/rlab10/OpenPCDet')
DATA_VARIANT = dataset_cfg.DATA_VARIANT
MODE = 'train' if dataset_cfg.DATA_SPLIT['train'] else 'val'

data_path_kitti = ROOT_DIR / 'data' / 'kitti' # raw data path
save_path_kitti = NOTEBOOK_DIR / 'kitti' / MODE
dflt_images_dir = save_path_kitti / 'bev_images_default'
dflt_annos_dir = save_path_kitti / 'bev_annos'

pkl_paths = {
    'train': {
        'default': data_path_kitti / dataset_cfg.PKL_TRAIN_DEFAULT,
        'densified': data_path_kitti / dataset_cfg.PKL_TRAIN_DENSIFIED,
        'upsampled': data_path_kitti / dataset_cfg.PKL_TRAIN_UPSAMPLED,
        'interpolated': data_path_kitti / dataset_cfg.PKL_TRAIN_INTERPOLATED,
    },
    'val': {
        'default': data_path_kitti / dataset_cfg.PKL_VAL_DEFAULT,
        'densified': data_path_kitti / dataset_cfg.PKL_VAL_DENSIFIED,
        'upsampled': data_path_kitti / dataset_cfg.PKL_VAL_UPSAMPLED,
        'interpolated': data_path_kitti / dataset_cfg.PKL_VAL_INTERPOLATED,
    }
}

pkl_path = pkl_paths[MODE][DATA_VARIANT]
compression_params = [cv2.IMWRITE_PNG_COMPRESSION, 1] # default compression value

dflt_images_exist = dflt_images_dir.exists() and any(dflt_images_dir.iterdir())
dflt_annos_exist = dflt_annos_dir.exists() and any(dflt_annos_dir.iterdir())

if dflt_images_exist and dflt_annos_exist:
    frame_ids_images = get_frame_ids(dflt_images_dir, '.png')
    frame_ids_annos = get_frame_ids(dflt_annos_dir, '.txt')
    valid_frame_ids = frame_ids_images & frame_ids_annos
    print(f"{len(valid_frame_ids)} valid frame_ids found")
else:
    valid_frame_ids = None
    print("Warning: bev_images_default and/or bev_annos does not exist or are empty. All samples will be used.")




def main(visualize_samples=False):
    print(f"Mode: {MODE}, Data Variant: {DATA_VARIANT}, Saving: {dataset_cfg.SAVE_FLAG}")
    print(f"Load: {pkl_path}")
    with open(pkl_path, 'rb') as f:
        data_list = pickle.load(f)
    
    if valid_frame_ids is not None and dataset_cfg.DEFAULT_SAMPLES_AS_ID:
        print(f"Generate BEV data for data variant {DATA_VARIANT} filtering on default frame_ids")
        data_list = [sample for sample in data_list if sample['frame_id'] in valid_frame_ids]
    else:
        print("No filtering applied, using all samples from pkl.")

    num_samples = len(data_list)
    # Original sample + 6 Augmentations else no augmentations
    num_augments = 7 if MODE == 'train' else 1 
    print(f"Samples: {num_samples}")
    print(f"Augmentations: {num_augments}")
    print(f"All: {num_samples * num_augments}")

    # DEBUG - single samples
    data_list = [data_list[4]]

    # main loop
    for sample_idx in tqdm(range(len(data_list)), desc="Samples"):
        for augment_idx in range(num_augments):
            sample = get_sample(data=data_list, sample_idx=sample_idx, augment_idx=augment_idx, mode=MODE, class_names=class_names)
            bev_image_bgr, _ = pointcloud3d_to_bevimage2d(points=sample['points'], cfg=dataset_cfg, num_slices=dataset_cfg.NUM_SLICES, 
                                                      filter_points=False)
            bev_image_bgr_vis = bev_image_bgr.copy()
            
            valid_bev_boxes, valid_indices = [], []
            if 'gt_boxes' in sample:
                for i in range(len(sample['gt_boxes'])):
                    boxes3d = sample['gt_boxes'][i]
                    bev_box = boxes3d_lidar_to_rotated_bev_boxes(
                        bev_img=bev_image_bgr,
                        bev_image_height=dataset_cfg.BEV_IMAGE_HEIGHT,
                        bev_image_width=dataset_cfg.BEV_IMAGE_WIDTH,
                        bev_res=dataset_cfg.CELL_SIZE,
                        boxes3d=boxes3d,
                        min_points=5
                    )                     
                    if bev_box is not None:
                        draw_bev_boxes(bev_image=bev_image_bgr_vis, bev_box=bev_box[:-1], cls_idx=int(bev_box[-1]), box_colormap=box_colormap, 
                                    thickness=1, obj_direction_color=(0,255,255))
                        valid_bev_boxes.append(bev_box)
                        valid_indices.append(i)

            if visualize_samples:
                key = show_bev_image_preview(bev_image=bev_image_bgr_vis, BGR2RGB=True, win_size=(800, 800))
            else:
                key = None

            if valid_bev_boxes and dataset_cfg.SAVE_FLAG:
                save_bev_images_and_boxes(save_path=save_path_kitti, frame_id=sample['frame_id'], bev_image_bgr=bev_image_bgr, 
                                        valid_bev_boxes=valid_bev_boxes, augment_idx=augment_idx, compression=compression_params,
                                        normalize_coords=dataset_cfg.NORM_BOX_COORDS, data_variant=DATA_VARIANT, mode=MODE)
                
                if MODE == 'val' and dataset_cfg.SAVE_FLAG and DATA_VARIANT == 'default':
                    generate_groundtruth_dicts_bev(sample, valid_indices, output_path=save_path_kitti / 'val_gt_data', frame_id=sample['frame_id'])

            if key == ord('q'):
                break
        if key == ord('q'):
            break
    cv2.destroyAllWindows()
    print(sample['frame_id'])

if __name__ == '__main__':
    main(visualize_samples=False)

In [ ]:
import os

# bev_dirs = [
#     ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_images_default', '.png'),
#     ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_images_densified', '.png'),
#     ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_images_interpolated', '.png'),
#     ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_images_upsampled', '.png'),
#     ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/bev_annos', '.txt'),
#     ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/kitti/val/val_gt_data', '.txt')
# ]


# bev_dirs = [
#     ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/zod/val/bev_images_default', '.png'),
#     ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/zod/val/bev_annos', '.txt'),
#     ('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/zod/val/val_gt_data', '.txt')
# ]

# Extrahiere den Dateistamm (ohne Endung)
def get_stem_set(bev_dir, ext):
    return set(os.path.splitext(f)[0] for f in os.listdir(bev_dir) if f.endswith(ext))

stem_sets = []
for bev_dir, ext in bev_dirs:
    stems = get_stem_set(bev_dir, ext)
    stem_sets.append(stems)
    print(f"{bev_dir}: {len(stems)} Dateien (Stamm)")

all_stems = set.union(*stem_sets)
common_stems = set.intersection(*stem_sets)
not_in_all = all_stems - common_stems

print("\nDateistämme, die NICHT in allen Verzeichnissen vorkommen:")
for stem in sorted(not_in_all):
    missing = [os.path.basename(d[0]) for d, s in zip(bev_dirs, stem_sets) if stem not in s]
    print(f"{stem} fehlt in: {', '.join(missing)}")

## Convert ZOD Point Cloud to BEV image

In [ ]:
from pathlib import Path
from easydict import EasyDict
from tqdm import tqdm
import pickle
import yaml
import cv2

from pcdet.utils.bev_utils import get_sample, pointcloud3d_to_bevimage2d, show_bev_image_preview,  generate_groundtruth_dicts_bev
from pcdet.utils.bev_utils import boxes3d_lidar_to_rotated_bev_boxes, draw_bev_boxes, save_bev_images_and_boxes


dataset_cfg = EasyDict(yaml.safe_load(open('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/dataset_configs/zod_dataset_bev_custom.yaml')))
class_names = ['Car', 'Pedestrian', 'Cyclist']
file_path = '/home/rlab10/OpenPCDet/lidar2bev/master_thesis/lidar3d_to_bev2d.ipynb' 
NOTEBOOK_DIR = (Path(file_path).resolve().parent).resolve()
ROOT_DIR = Path('/home/rlab10/OpenPCDet')
DATA_VARIANT = dataset_cfg.DATA_VARIANT
MODE = 'train' if dataset_cfg.DATA_SPLIT['train'] else 'val'

data_path_zod = ROOT_DIR / 'data' / 'zod' # raw data path
save_path_zod = NOTEBOOK_DIR / 'zod' / MODE

box_colormap = [
    [1, 1, 1], # not assigned
    [0, 1, 0], # Car (green)
    [1, 0, 1], # Pedestrian (violet)
    [0, 1, 1], # Cyclist (yellow)
]

pkl_paths = {
    'train': {
        'default': data_path_zod / dataset_cfg.PKL_TRAIN_DEFAULT,
    },
    'val': {
        'default': data_path_zod / dataset_cfg.PKL_VAL_DEFAULT,
    }
}

pkl_path = pkl_paths[MODE][DATA_VARIANT]
compression_params = [cv2.IMWRITE_PNG_COMPRESSION, 1] # default compression value



def main(visualize_samples=False):
    print(f"Mode: {MODE}, Data Variant: {DATA_VARIANT}, Saving: {dataset_cfg.SAVE_FLAG}")
    print(f"Load: {pkl_path}")
    with open(pkl_path, 'rb') as f:
        data_list = pickle.load(f)

    num_samples = len(data_list)
    # Original sample + 6 Augmentations else no augmentations
    num_augments = 7 if MODE == 'train' else 1 
    print(f"Samples: {num_samples}")
    print(f"Augmentations: {num_augments}")
    print(f"All: {num_samples * num_augments}")

    # DEBUG - single samples
    data_list = [data_list[0]]

    # main loop
    for sample_idx in tqdm(range(len(data_list)), desc="Samples"):
        for augment_idx in tqdm(range(num_augments), disable=True):
            sample = get_sample(data=data_list, sample_idx=sample_idx, augment_idx=augment_idx, mode=MODE, class_names=class_names)
            bev_image_bgr, _ = pointcloud3d_to_bevimage2d(points=sample['points'], cfg=dataset_cfg, num_slices=dataset_cfg.NUM_SLICES, 
                                                      filter_points=False)
            bev_image_bgr_vis = bev_image_bgr.copy()
            
            valid_bev_boxes, valid_indices = [], []
            if 'gt_boxes' in sample:
                for i in range(len(sample['gt_boxes'])):
                    boxes3d = sample['gt_boxes'][i]
                    bev_box = boxes3d_lidar_to_rotated_bev_boxes(
                        bev_img=bev_image_bgr,
                        bev_image_height=dataset_cfg.BEV_IMAGE_HEIGHT,
                        bev_image_width=dataset_cfg.BEV_IMAGE_WIDTH,
                        bev_res=dataset_cfg.CELL_SIZE,
                        boxes3d=boxes3d,
                        min_points=5
                    )
                    if bev_box is not None:
                        draw_bev_boxes(bev_image=bev_image_bgr_vis, bev_box=bev_box[:-1], cls_idx=int(bev_box[-1]), box_colormap=box_colormap, 
                                       thickness=1, obj_direction_color=(0,255,255))
                        valid_bev_boxes.append(bev_box)
                        valid_indices.append(i)

            if visualize_samples:
                key = show_bev_image_preview(bev_image=bev_image_bgr_vis, BGR2RGB=True, win_size=(800, 800))
            else:
                key = None

            if valid_bev_boxes and dataset_cfg.SAVE_FLAG:
                save_bev_images_and_boxes(save_path=save_path_zod, frame_id=sample['frame_id'], bev_image_bgr=bev_image_bgr, 
                                          valid_bev_boxes=valid_bev_boxes, augment_idx=augment_idx, compression=compression_params,
                                          normalize_coords=dataset_cfg.NORM_BOX_COORDS, data_variant=DATA_VARIANT, mode=MODE)
                
                if MODE == 'val' and dataset_cfg.SAVE_FLAG:
                    generate_groundtruth_dicts_bev(sample, valid_indices, output_path=save_path_zod / 'val_gt_data', frame_id=sample['frame_id'])

            if key == ord('q'):
                break
        if key == ord('q'):
            break
    cv2.destroyAllWindows()
    print(sample['frame_id'])
    

if __name__ == '__main__':
    main(visualize_samples=False)